# Webbskrapning

Webbskrapning (_eng. web scraping_) går ut på att hämta information från sidor på nätet

Motiveringar:

- När data du behöver inte finns färdigt samlad
- Forskningssyfte för att skaffa information
- Kommersiellt syfte för att automatisera marknadsföring eller analysera kunders val på nätet

Lagligt?

Allmänt tillgänglig data

- Användaren har gjort datan offentlig
- Det krävs inget konto för att nå datan
- Sidan är inte blockerad av en `robots.txt` fil, exempelvis Yle har en sådan https://yle.fi/robots.txt. Vi tar och ser vad den innehåller:

In [1]:
import requests

robots = requests.get('https://yle.fi/robots.txt')
print(sida.text)

NameError: name 'sida' is not defined

- `User-agent: *` i filen innebär att restriktionerna gäller alla sorters robotar
- `Disallow: /eurovision/` innebär att robotar inte får skrapa https://yle.fi/eurovision/

## Dagens nyheter

Vårt mål är här att automatiskt få dagens nyheter listade.

In [2]:
from bs4 import BeautifulSoup

yle = requests.get('https://svenska.yle.fi/')
soppa = BeautifulSoup(yle.content, 'html.parser')

In [3]:
titel = soppa.find('title')
print(titel)

<title>Svenska Yle</title>


In [4]:
sidhuvud = soppa.find('header')

for lank in sidhuvud.find_all('a', text=True):
  print(f"{lank.text:20s}{lank.get('href')}")

svenska.yle.fi      https://svenska.yle.fi
Svenska Yle         https://svenska.yle.fi
Arenan              https://arenan.yle.fi/tv
Ämnen               https://svenska.yle.fi/lista/amnessidor
Svenska Yle         https://svenska.yle.fi
Arenan              https://arenan.yle.fi/tv
Inrikes             https://svenska.yle.fi/inrikes
Sport               https://svenska.yle.fi/sport
Utrikes             https://svenska.yle.fi/utrikes
Kultur              https://svenska.yle.fi/kultur
Familj              https://svenska.yle.fi/familj
Kunskap             https://svenska.yle.fi/vetamix
Huvudstadsregionen  https://svenska.yle.fi/huvudstadsregionen
Västnyland          https://svenska.yle.fi/vastnyland
Åboland             https://svenska.yle.fi/aboland
Österbotten         https://svenska.yle.fi/osterbotten
Östnyland           https://svenska.yle.fi/ostnyland
Kontakta oss        https://svenska.yle.fi/respons
Radio-guide         https://arenan.yle.fi/radio/guide
TV-guide            https://arenan.yle.

Vi söker alla länkar

In [5]:
yle_lankar = soppa.find_all('a')

In [6]:
print(yle_lankar[2].prettify())

<a href="https://svenska.yle.fi" id="yle-header-main-link--svenska-yle" tabindex="0">
 Svenska Yle
</a>



In [7]:
print(soppa.find('h3').prettify())

<h3 class="js-ydd-card-title flex txt-16">
 Senaste nytt om coronaviruset
</h3>



In [8]:
for yle_lank in yle_lankar:
  rubrik = yle_lank.find('h3')
  if rubrik:
    print('Rubrik:', rubrik.text.strip())
    print('Länk:', yle_lank.get('href'))
    break

Rubrik: Senaste nytt om coronaviruset
Länk: https://svenska.yle.fi/coronavirus


href egenskapen anger länkens destination

In [9]:
rubriker = []
lankar = []

for yle_lank in yle_lankar:
  rubrik = yle_lank.find('h3')
  if rubrik:
    rubriker.append(rubrik.text.strip())
    lankar.append(yle_lank.get('href'))

In [10]:
import pandas as pd

nyheter = pd.DataFrame({'Rubrik': rubriker, 'Länk': lankar})

In [11]:
pd.set_option('display.max_colwidth', None)
nyheter.head(10)

,Rubrik,Länk
0,Senaste nytt om coronaviruset,https://svenska.yle.fi/coronavirus
1,IT-företagaren Kaj Arnö: Hårt arbete och kloka beslut eller bondtur att vi lyckades sälja IT-firman för en miljard dollar?,https://svenska.yle.fi/artikel/2020/07/01/it-foretagaren-kaj-arno-hart-arbete-och-kloka-beslut-eller-bondtur-att-vi
2,Lyssna på Kaj Arnös sommarprat,https://arenan.yle.fi/1-50548527
3,Tiotusentals hushåll utan el efter stormen Päivö - på onsdag väntar häftiga åskväder,https://svenska.yle.fi/artikel/2020/07/01/tiotusentals-hushall-utan-el-efter-stormen-paivo-pa-onsdag-vantar-haftiga
4,Stormen Päivö drog in över Borgå - gatubrunnar svämmade över och fyllde källare med vatten,https://svenska.yle.fi/artikel/2020/06/30/stormen-paivo-drog-in-over-borga-gatubrunnar-svammade-over-och-fyllde-kallare-med
5,MT:s gallup: Haavisto och Halla-aho tappar i popularitet som presidentkandidater – Marin upp på tredje plats,https://svenska.yle.fi/artikel/2020/07/01/mts-gallup-haavisto-och-halla-aho-tappar-i-popularitet-som-presidentkandidater
6,"USA:s Fauci: Hela landet hotas av snabb coronaspridning - ""Skulle inte vara förvånad om vi är uppe i över 100 000 fall per dag""",https://svenska.yle.fi/artikel/2020/07/01/usas-fauci-hela-landet-hotas-av-snabb-coronaspridning-skulle-inte-vara-forvanad
7,Kommentar: Så länge ryssarna sväljer maktfullkomlighet och valfusk sitter Putin stadigt kvar,https://svenska.yle.fi/artikel/2020/07/01/kommentar-sa-lange-ryssarna-svaljer-maktfullkomlighet-och-valfusk-sitter-putin
8,"Coronakrisen i centrum när Tyskland tar över EU-ordförandeklubban, juli inleds med uppluckrade restriktioner för utomhusevenemang – läs det senaste om coronaviruset här",https://svenska.yle.fi/coronavirus
9,Hjärnskadade Irina föreläser ibland om sina erfarenheter - i FPA:s ögon är hon arbetsför och behöver inget stöd,https://svenska.yle.fi/artikel/2020/07/01/hjarnskadade-irina-forelaser-ibland-om-sina-erfarenheter-i-fpas-ogon-ar-hon


## Konsol

## Quiz